In [1]:
%pip install -qU mistralai langchain-core langchain langchain-mistralai gradio

Note: you may need to restart the kernel to use updated packages.


In [2]:
import getpass, os
os.environ["MISTRAL_API_KEY"] = getpass.getpass("Mistral AI API Key:")

Mistral AI API Key: ········


In [ ]:
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
import gradio as gr
import time

model = "mistral-large-latest"

with gr.Blocks(theme=gr.themes.Glass()) as demo:
    progress_textbox = gr.components.Textbox(interactive=False, visible=True, autoscroll=True, show_label=False)
    empty_html_template = """
    <div style='height: 20px; width: 100%; background-color: pink;'>{content}</div>
    """
    html_area = gr.HTML(empty_html_template.format(content=""))
    chatbot = gr.Chatbot(show_label=False)
    msg = gr.Textbox(show_label=False)
    clear = gr.Button("Clear")

    def user(user_message, history):
        return "", history + [[user_message, None]]

    statuses = [
        "step I success",
        "step II success",
        "step III failure",
        "step III success",
        "step IV success",
        "step V success"
    ]

    def process():
        for i in range(len(statuses)+1):
            time.sleep(0.5)
            yield '\n'.join(statuses[0:i])

    def html_log():
        for i in range(len(statuses)+1):
            time.sleep(0.5)
            yield empty_html_template.format(content='<br />'.join(statuses[0:i]))

    def bot(history):
        history_langchain_format = []

        # Convert the history to the format expected by the ChatMistralAI client
        for entry in history:
            if len(entry) == 2:
                human, ai = entry
                history_langchain_format.append(ChatMessage(role="user", content=human))
                if ai is not None:
                    history_langchain_format.append(ChatMessage(role="assistant", content=ai))

        from mistralai.client import MistralClient
        client = MistralClient()
        response = client.chat_stream(
            model=model,
            messages=history_langchain_format,
        )

        history[-1][1] = ""
        for chunk in response:
            if chunk.choices[0].delta.content is not None:
                history[-1][1] += chunk.choices[0].delta.content
                yield history

    action_step = msg.submit(
            user, [msg, chatbot], [msg, chatbot], queue=False
        ).then(
          # clear progress_textbox
          lambda: None, None, progress_textbox, queue=False
        ).then(
          # clear html_area
          lambda: empty_html_template.format(content=""), None, html_area, queue=False
        ).then(
            process, None, progress_textbox
        ).then(
            html_log, None, html_area
        )


    # final response to user query
    action_step.then(
        bot, chatbot, chatbot
    )

    # clear
    clear.click(lambda: None, None, chatbot, queue=False).then(
       # clear progress_textbox
       lambda: None, None, progress_textbox, queue=False
    ).then(
       # clear html_area
       lambda: empty_html_template.format(content=""), None, html_area, queue=False
    )

demo.queue()
demo.launch(debug=True)


/Users/nossa/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
